<a href="https://colab.research.google.com/github/mar-i0/AI-Notebooks/blob/main/Custom_LLM_Model_for_INDEXING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Instalamos las dependencias necesarias:



In [1]:
!pip install torch
!pip install langchain
!pip install llama_index
!pip install transformers
!pip install typing
!pip install PyPDF2
%pip install accelerate>=0.12.0 transformers[torch]==4.25.1



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


2. Importamos las librerias necesarias

In [2]:
import torch
from langchain.llms.base import LLM
from llama_index import SimpleDirectoryReader, LangchainEmbedding, GPTListIndex, PromptHelper
from llama_index import GPTSimpleVectorIndex
from llama_index import LLMPredictor, ServiceContext
from transformers import pipeline
from typing import Optional, List, Mapping, Any


3. Definimos las variables necesarias

In [3]:
# define prompt helper
# set maximum input size
max_input_size = 2048
# set number of output tokens
num_output = 256
# set maximum chunk overlap
max_chunk_overlap = 20
prompt_helper = PromptHelper(max_input_size, num_output, max_chunk_overlap)


4. Definimos las funciones personalizadas necesarias

In [4]:
!nvidia-smi

Mon Apr 17 20:17:23 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    44W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
class CustomLLM(LLM):
    #model_name = "facebook/opt-iml-max-30b"
    model_name = "facebook/opt-iml-1.3b"
    pipeline = pipeline("text-generation", model=model_name, device="cuda:0", model_kwargs={"torch_dtype":torch.bfloat16})

    #model_name = "databricks/dolly-v2-12b"
    #model_name = "lmsys/vicuna-7b-delta-v0"
    #pipeline = pipeline(model=model_name, torch_dtype=torch.bfloat16, trust_remote_code=True, device_map="auto", device=0)
    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        prompt_length = len(prompt)
        response = self.pipeline(prompt, max_new_tokens=num_output)[0]["generated_text"]

        # only return newly generated tokens
        return response[prompt_length:]

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        return {"name_of_model": self.model_name}

    @property
    def _llm_type(self) -> str:
        return "custom"

5. Ejecutamos

In [13]:
# define our LLM
llm_predictor = LLMPredictor(llm=CustomLLM())

service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, prompt_helper=prompt_helper)

# Load the your data
documents = SimpleDirectoryReader('./sample_data').load_data()

index = GPTListIndex.from_documents(documents, service_context=service_context)
##index = GPTSimpleVectorIndex.from_documents(documents)
print ("Hecho todo OK")


Hecho todo OK


In [14]:
## Query and print response
response = index.query("Abaixadors")
print(response)


/usr/local/lib/python3.9/dist-packages/transformers/pipelines/base.py:1043: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



------------
The original question is as follows: What is the name of the pokemon that is the most common in the game?
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
False
700, Sylveon, Fairy, nan, 525, 95, 65, 65, 110, 130, 60, 6, False
701, Hawlucha, Fighting, Flying, 500, 78, 92, 75, 74, 63, 118, 6, False
702, Dedenne, Electric, Fairy, 431, 67, 58, 57, 81, 67, 101, 6, False
703, Carbink, Rock, Fairy, 500, 50, 50, 150, 50, 150, 50, 6, False
704, Goomy, Dragon, nan, 300, 45, 50, 35, 55, 75, 40, 6, False
704, GoomySmall Size, Dragon, 450, 65, 90, 120, 130, 70, 6, False
704, GoomyLarge Size, Dragon, 450, 65, 90, 120, 130, 70, 6, False
705, Sliggoo, Dragon, nan, 452, 68, 75


In [16]:
print(index.query("What is the HP of Fairy?"))

 The original answer is as follows: The original HP of Fairy is the HP of the element Fairy.


In [ ]:
print(index.query("Tell 2 legendary pokemon"))